# CITS 影像分析與能帶/能譜圖測試 / CITS Image Analysis with Band/Spectrum Testing

**作者 / Author**: Odindino  
**測試目標**:

1. 📂 載入 CITS 檔案
2. 🎛️ 使用 slider 調整偏壓查看不同切片
3. 📍 選擇點位生成能譜圖 (Energy Spectrum)
4. 📈 拉線生成能帶圖 (Band Profile)

**使用方式**: 執行每個 cell，使用 slider 和交互式工具進行分析

## 📦 模組載入 / Module Loading

In [2]:
import sys
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output

# 設置 Plotly
pyo.init_notebook_mode(connected=True)
pio.renderers.default = "notebook"

# 添加後端路徑
backend_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

# 導入 KEEN 模組
from core.experiment_session import ExperimentSession

print("✅ 模組載入完成")
print(f"📂 工作目錄: {os.getcwd()}")
print(f"🐍 後端路徑: {backend_path}")

✅ 模組載入完成
📂 工作目錄: /Users/yangziliang/Git-Projects/keen/backend/test/notebooks
🐍 後端路徑: /Users/yangziliang/Git-Projects/keen/backend


## ⚙️ 設定區域 / Configuration Area

In [3]:
# ============================================================================
# 📝 檔案路徑設定 / File Path Settings
# ============================================================================

TXT_FILE_PATH = '../../../testfile/20250521_Janus Stacking SiO2_13K_113.txt'

# 全局變數初始化
session = None
cits_analyzer = None
cits_data = None
current_bias_index = 0
output_plot = None
profile_plot = None

print("🔧 當前設定 / Current Settings:")
print(f"📂 TXT 檔案: {TXT_FILE_PATH}")
print(f"📁 TXT 檔案存在: {os.path.exists(TXT_FILE_PATH)}")

🔧 當前設定 / Current Settings:
📂 TXT 檔案: ../../../testfile/20250521_Janus Stacking SiO2_13K_113.txt
📁 TXT 檔案存在: True


## 📂 步驟 1: 載入檔案 / Step 1: Load Files

In [4]:
def load_cits_files():
    """載入 CITS 檔案"""
    global session, cits_analyzer, cits_data
    
    try:
        print("📂 開始載入檔案...")
        
        # 初始化會話
        session = ExperimentSession(TXT_FILE_PATH)
        print(f"✅ 實驗載入成功: {session.experiment_name}")
        
        # 列出可用的 CITS 檔案
        cits_files = session.get_cits_files()
        print(f"\n📋 可用的 CITS 檔案: {cits_files}")
        
        if not cits_files:
            print("❌ 沒有找到 CITS 檔案")
            return False
            
        # 載入第一個 CITS 檔案
        cits_key = cits_files[0]
        print(f"\n🔄 載入 CITS 檔案: {cits_key}")
        
        cits_proxy = session.get_file(cits_key)
        cits_data = cits_proxy.data
        cits_analyzer = cits_proxy.analyzer
        
        # 顯示 CITS 資訊
        print(f"✅ CITS 檔案載入成功")
        print(f"📊 資料形狀: {cits_data.cits_data.shape} (height, width, bias)")
        print(f"📏 掃描範圍: {cits_data.x_range:.1f} × {cits_data.y_range:.1f} nm")
        print(f"⚡ 偏壓範圍: {cits_data.bias_range[0]:.3f} ~ {cits_data.bias_range[-1]:.3f} V")
        print(f"⚡ 偏壓點數: {len(cits_data.bias_range)}")
        
        return True
        
    except Exception as e:
        print(f"❌ 載入失敗: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return False

# 執行載入
load_success = load_cits_files()
if load_success:
    print("\n🎉 CITS 檔案載入成功! 繼續下一步。")
else:
    print("\n💡 請檢查檔案路徑")

📂 開始載入檔案...
✅ 實驗載入成功: Unknown

📋 可用的 CITS 檔案: ['20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix']

🔄 載入 CITS 檔案: 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
✅ CITS 檔案載入成功
❌ 載入失敗: 'CitsData' object has no attribute 'cits_data'
Traceback (most recent call last):
  File "/var/folders/s8/x7qjkqzs44lg2tdb36y2xh680000gn/T/ipykernel_94878/1965488641.py", line 30, in load_cits_files
    print(f"📊 資料形狀: {cits_data.cits_data.shape} (height, width, bias)")
                          ^^^^^^^^^^^^^^^^^^^
AttributeError: 'CitsData' object has no attribute 'cits_data'


💡 請檢查檔案路徑


/Users/yangziliang/Git-Projects/keen/backend/core/parsers/dat_parser.py:84: DtypeWarning:

Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.



def extract_point_spectrum(x, y):
    """提取單點能譜"""
    if cits_data is None:
        print("❌ 請先載入 CITS 檔案")
        return None
        
    try:
        # 直接從 CITS 數據提取指定點的能譜
        h, w, n_bias = cits_data.cits_data.shape
        
        # 檢查座標範圍
        if not (0 <= x < w and 0 <= y < h):
            print(f"❌ 座標超出範圍: ({x}, {y}), 最大範圍: ({w-1}, {h-1})")
            return None
            
        # 提取該點的所有偏壓數據
        spectrum = cits_data.cits_data[y, x, :]  # 注意: [y, x, bias]
        bias_values = cits_data.bias_range
        
        # 計算 dI/dV (簡單數值微分)
        dIdV = np.gradient(spectrum, bias_values)
        
        # 計算統計數據
        stats = {
            'mean_current': np.mean(spectrum),
            'max_current': np.max(spectrum),
            'min_current': np.min(spectrum),
            'std_current': np.std(spectrum),
            'current_range': np.max(spectrum) - np.min(spectrum)
        }
        
        # 簡單的能隙檢測 (找到 dI/dV 最小值)
        didv_min_idx = np.argmin(np.abs(dIdV))
        gap_voltage = bias_values[didv_min_idx]
        stats['gap_voltage'] = gap_voltage
        
        # 創建圖表
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=[f'I-V 曲線 @ ({x}, {y})', 'dI/dV 曲線'],
            vertical_spacing=0.15
        )
        
        # I-V 曲線
        fig.add_trace(go.Scatter(
            x=bias_values,
            y=spectrum,
            mode='lines+markers',
            name='I-V',
            line=dict(color='blue', width=2),
            marker=dict(size=4)
        ), row=1, col=1)
        
        # dI/dV 曲線
        fig.add_trace(go.Scatter(
            x=bias_values,
            y=dIdV,
            mode='lines',
            name='dI/dV',
            line=dict(color='red', width=2)
        ), row=2, col=1)
        
        # 標記能隙位置
        fig.add_vline(
            x=gap_voltage, 
            line_dash="dash", 
            line_color="green",
            annotation_text=f"Gap: {gap_voltage:.3f}V",
            row=2, col=1
        )
        
        fig.update_xaxes(title_text="Bias (V)", row=1, col=1)
        fig.update_yaxes(title_text="Current (pA)", row=1, col=1)
        fig.update_xaxes(title_text="Bias (V)", row=2, col=1)
        fig.update_yaxes(title_text="dI/dV (pA/V)", row=2, col=1)
        
        fig.update_layout(
            title=f'點能譜分析 @ ({x}, {y})',
            height=700,
            showlegend=False
        )
        
        fig.show()
        
        # 顯示統計
        print(f"\n📊 能譜統計:")
        print(f"   平均電流: {stats['mean_current']:.3f} pA")
        print(f"   電流範圍: {stats['min_current']:.3f} ~ {stats['max_current']:.3f} pA")
        print(f"   標準差: {stats['std_current']:.3f} pA")
        print(f"   估算能隙: {stats['gap_voltage']:.3f} V")
        
        return {
            'bias': bias_values,
            'current': spectrum,
            'dIdV': dIdV,
            'stats': stats,
            'coordinates': (x, y)
        }
        
    except Exception as e:
        print(f"❌ 錯誤: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

# 測試幾個點
if load_success:
    print("📍 測試單點能譜提取:")
    print("=" * 50)
    
    # 測試中心點
    h, w, _ = cits_data.cits_data.shape
    center_x, center_y = w // 2, h // 2
    
    print(f"\n1️⃣ 中心點 ({center_x}, {center_y}):")
    spectrum1 = extract_point_spectrum(center_x, center_y)
    
    # 測試角落點
    print(f"\n2️⃣ 角落點 (10, 10):")
    spectrum2 = extract_point_spectrum(10, 10)
else:
    print("⚠️ 請先成功載入 CITS 檔案")

In [ ]:
def extract_line_profile(start_point, end_point, num_points=50):
    """沿著線提取能帶剖面"""
    if cits_data is None:
        print("❌ 請先載入 CITS 檔案")
        return None
        
    try:
        print(f"📏 提取能帶剖面: {start_point} → {end_point}")
        
        h, w, n_bias = cits_data.cits_data.shape
        y0, x0 = start_point
        y1, x1 = end_point
        
        # 檢查座標範圍
        if not (0 <= x0 < w and 0 <= x1 < w and 0 <= y0 < h and 0 <= y1 < h):
            print(f"❌ 座標超出範圍: 最大範圍 ({w-1}, {h-1})")
            return None
        
        # 生成線上的採樣點 (Bresenham 算法)
        def bresenham_line(x0, y0, x1, y1):
            \"\"\"Bresenham 線算法\"\"\"
            points = []
            dx = abs(x1 - x0)
            dy = abs(y1 - y0)
            x, y = x0, y0
            x_inc = 1 if x1 > x0 else -1
            y_inc = 1 if y1 > y0 else -1
            error = dx - dy
            
            while True:
                points.append((x, y))
                if x == x1 and y == y1:
                    break
                if error * 2 > -dy:
                    error -= dy
                    x += x_inc
                if error * 2 < dx:
                    error += dx
                    y += y_inc
            return points
        
        # 獲取線上的所有點
        line_points = bresenham_line(x0, y0, x1, y1)
        
        # 提取沿線的所有能譜數據
        line_spectra = []
        for x, y in line_points:
            spectrum = cits_data.cits_data[y, x, :]
            line_spectra.append(spectrum)
        
        intensity_map = np.array(line_spectra)  # (n_points, n_bias)
        
        # 計算物理座標
        pixel_scale_x = cits_data.x_range / w
        pixel_scale_y = cits_data.y_range / h
        
        # 計算沿線的距離
        distances = []
        total_distance = 0
        distances.append(0)
        
        for i in range(1, len(line_points)):
            x_prev, y_prev = line_points[i-1]
            x_curr, y_curr = line_points[i]
            dx = (x_curr - x_prev) * pixel_scale_x
            dy = (y_curr - y_prev) * pixel_scale_y
            step_distance = np.sqrt(dx**2 + dy**2)
            total_distance += step_distance
            distances.append(total_distance)
        
        # 計算統計數據
        avg_spectrum = np.mean(intensity_map, axis=0)
        std_spectrum = np.std(intensity_map, axis=0)
        
        # 創建 2D 能帶圖
        fig = make_subplots(
            rows=3, cols=1,
            row_heights=[0.5, 0.25, 0.25],
            subplot_titles=[
                '能帶圖 (Position vs Bias)', 
                '空間平均能譜', 
                '能譜標準差'
            ],
            vertical_spacing=0.12
        )
        
        # 1. 能帶圖 (2D heatmap)
        bias_values = cits_data.bias_range
        
        fig.add_trace(
            go.Heatmap(
                x=distances,
                y=bias_values,
                z=intensity_map.T,  # 轉置以匹配座標
                colorscale='RdBu_r',
                colorbar=dict(title='Current (pA)', y=0.77, len=0.4),
                hovertemplate='Distance: %{x:.2f} nm<br>Bias: %{y:.3f} V<br>Current: %{z:.3f} pA<extra></extra>'
            ),
            row=1, col=1
        )
        
        # 2. 空間平均能譜
        fig.add_trace(
            go.Scatter(
                x=bias_values,
                y=avg_spectrum,
                mode='lines',
                line=dict(color='blue', width=2),
                name='平均能譜'
            ),
            row=2, col=1
        )
        
        # 3. 標準差
        fig.add_trace(
            go.Scatter(
                x=bias_values,
                y=std_spectrum,
                mode='lines',
                line=dict(color='red', width=2),
                name='標準差'
            ),
            row=3, col=1
        )
        
        # 更新布局
        fig.update_xaxes(title_text=\"Position (nm)\", row=1, col=1)
        fig.update_yaxes(title_text=\"Bias (V)\", row=1, col=1)
        fig.update_xaxes(title_text=\"Bias (V)\", row=2, col=1)
        fig.update_yaxes(title_text=\"Current (pA)\", row=2, col=1)
        fig.update_xaxes(title_text=\"Bias (V)\", row=3, col=1)
        fig.update_yaxes(title_text=\"Std (pA)\", row=3, col=1)
        
        fig.update_layout(
            title=f'能帶剖面分析: {start_point} → {end_point}',
            height=900,
            showlegend=False
        )
        
        fig.show()
        
        # 顯示統計資訊
        line_length = distances[-1]
        print(f\"\\n📊 能帶剖面統計:\")
        print(f\"   剖面長度: {line_length:.2f} nm\")
        print(f\"   採樣點數: {len(line_points)}\")
        print(f\"   平均電流範圍: {np.min(avg_spectrum):.3f} ~ {np.max(avg_spectrum):.3f} pA\")
        print(f\"   最大標準差: {np.max(std_spectrum):.3f} pA\")
        
        return {
            'intensity_map': intensity_map,
            'line_length': line_length,
            'positions': distances,
            'line_points': line_points,
            'average_spectrum': avg_spectrum,
            'std_spectrum': std_spectrum,
            'bias_values': bias_values,
            'stats': {
                'line_length': line_length,
                'n_points': len(line_points),
                'min_avg_current': np.min(avg_spectrum),
                'max_avg_current': np.max(avg_spectrum),
                'max_std': np.max(std_spectrum)
            }
        }
        
    except Exception as e:
        print(f\"❌ 錯誤: {str(e)}\")
        import traceback
        print(traceback.format_exc())
        return None

# 測試能帶剖面
if load_success:
    print(\"📈 測試能帶剖面提取:\")
    print(\"=\" * 50)
    
    h, w, _ = cits_data.cits_data.shape
    
    # 水平剖面 (中間)
    print(\"\\n1️⃣ 水平剖面 (中間):\")
    profile1 = extract_line_profile((h//2, 0), (h//2, w-1))
    
    # 對角線剖面
    print(\"\\n2️⃣ 對角線剖面:\")
    profile2 = extract_line_profile((0, 0), (h-1, w-1))
else:
    print(\"⚠️ 請先成功載入 CITS 檔案\")

## 📍 步驟 3: 單點能譜分析 / Step 3: Single Point Spectrum Analysis

In [ ]:
# 計算和顯示 dI/dV map
def calculate_didv_map(bias_index=None):
    """計算特定偏壓的 dI/dV map 或整個 dI/dV 數據"""
    if cits_data is None:
        return None
        
    try:
        # 對每個像素計算 dI/dV
        h, w, n_bias = cits_data.cits_data.shape
        didv_data = np.zeros_like(cits_data.cits_data)
        
        # 沿偏壓軸計算梯度
        for y in range(h):
            for x in range(w):
                spectrum = cits_data.cits_data[y, x, :]
                didv = np.gradient(spectrum, cits_data.bias_range)
                didv_data[y, x, :] = didv
        
        if bias_index is not None:
            return didv_data[:, :, bias_index]
        else:
            return didv_data
            
    except Exception as e:
        print(f"❌ dI/dV 計算失敗: {str(e)}")
        return None

def show_cits_with_markers(points=None, lines=None):
    """顯示 CITS 影像並標記選擇的點和線"""
    if cits_data is None:
        print("❌ 請先載入 CITS 檔案")
        return
    
    # 使用當前偏壓切片
    slice_data = cits_data.cits_data[:, :, current_bias_index]
    bias_value = cits_data.bias_range[current_bias_index]
    
    fig = go.Figure()
    
    # 添加 CITS 影像
    fig.add_trace(go.Heatmap(
        z=slice_data,
        colorscale='RdBu_r',
        showscale=True,
        colorbar=dict(title='Current (pA)')
    ))
    
    # 添加點標記
    if points:
        for i, (x, y) in enumerate(points):
            fig.add_trace(go.Scatter(
                x=[x], y=[y],
                mode='markers',
                marker=dict(size=10, color='yellow', symbol='x'),
                name=f'點 {i+1}',
                showlegend=True
            ))
    
    # 添加線標記
    if lines:
        for i, ((y1, x1), (y2, x2)) in enumerate(lines):
            fig.add_trace(go.Scatter(
                x=[x1, x2], y=[y1, y2],
                mode='lines+markers',
                line=dict(color='lime', width=2),
                marker=dict(size=8),
                name=f'線 {i+1}',
                showlegend=True
            ))
    
    fig.update_layout(
        title=f'CITS 影像 @ {bias_value:.3f} V (標記點和線)',
        xaxis_title='X (pixels)',
        yaxis_title='Y (pixels)',
        width=700,
        height=700,
        yaxis=dict(scaleanchor=\"x\", scaleratio=1)
    )
    
    fig.show()

def show_didv_map(bias_index=None):
    """顯示 dI/dV map"""
    if bias_index is None:
        bias_index = current_bias_index
        
    didv_slice = calculate_didv_map(bias_index)
    if didv_slice is None:
        return
        
    bias_value = cits_data.bias_range[bias_index]
    
    fig = go.Figure()
    
    fig.add_trace(go.Heatmap(
        z=didv_slice,
        colorscale='RdYlBu_r',
        colorbar=dict(title='dI/dV (pA/V)'),
        hovertemplate='X: %{x}<br>Y: %{y}<br>dI/dV: %{z:.3f} pA/V<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'dI/dV Map @ {bias_value:.3f} V',
        xaxis_title='X (pixels)',
        yaxis_title='Y (pixels)',
        width=600,
        height=600,
        yaxis=dict(scaleanchor=\"x\", scaleratio=1)
    )
    
    fig.show()

# 測試標記功能
if load_success:
    print(\"🎨 顯示標記的 CITS 影像:\")
    
    h, w, _ = cits_data.cits_data.shape
    
    # 定義一些測試點和線
    test_points = [(w//4, h//4), (3*w//4, 3*h//4)]
    test_lines = [((h//2, 0), (h//2, w-1)), ((0, w//2), (h-1, w//2))]
    
    show_cits_with_markers(points=test_points, lines=test_lines)

# 顯示 dI/dV
if load_success:
    print(\"\\n📈 計算並顯示 dI/dV map:\")
    show_didv_map()
    print(f\"✅ dI/dV map 顯示完成\")

## 📈 步驟 4: 能帶圖分析 (拉線) / Step 4: Band Profile Analysis (Line Profile)

In [ ]:
def extract_line_profile(start_point, end_point, num_points=50):
    """沿著線提取能帶剖面"""
    if cits_analyzer is None:
        print("❌ 請先載入 CITS 檔案")
        return None
        
    try:
        print(f"📏 提取能帶剖面: {start_point} → {end_point}")
        
        # 提取線剖面
        result = cits_analyzer.extract_line_profile(start_point, end_point, num_points)
        
        if result['success']:
            profile_data = result['data']
            
            # 創建 2D 能帶圖
            fig = make_subplots(
                rows=2, cols=1,
                row_heights=[0.7, 0.3],
                subplot_titles=['能帶圖 (Position vs Bias)', '空間平均能譜'],
                vertical_spacing=0.15
            )
            
            # 1. 能帶圖 (2D heatmap)
            positions = profile_data['positions']
            bias_values = cits_data.bias_range
            intensity_map = profile_data['intensity_map']
            
            fig.add_trace(
                go.Heatmap(
                    x=positions,
                    y=bias_values,
                    z=intensity_map.T,  # 轉置以匹配座標
                    colorscale='RdBu_r',
                    colorbar=dict(title='Current (pA)', y=0.85, len=0.5)
                ),
                row=1, col=1
            )
            
            # 2. 空間平均能譜
            avg_spectrum = profile_data.get('average_spectrum', np.mean(intensity_map, axis=0))
            
            fig.add_trace(
                go.Scatter(
                    x=bias_values,
                    y=avg_spectrum,
                    mode='lines',
                    line=dict(color='blue', width=2),
                    name='平均能譜'
                ),
                row=2, col=1
            )
            
            # 更新布局
            fig.update_xaxes(title_text="Position (nm)", row=1, col=1)
            fig.update_yaxes(title_text="Bias (V)", row=1, col=1)
            fig.update_xaxes(title_text="Bias (V)", row=2, col=1)
            fig.update_yaxes(title_text="Current (pA)", row=2, col=1)
            
            fig.update_layout(
                title=f'能帶剖面分析: {start_point} → {end_point}',
                height=800,
                showlegend=False
            )
            
            fig.show()
            
            # 顯示統計資訊
            print(f"\n📊 能帶剖面統計:")
            print(f"   剖面長度: {profile_data['line_length']:.2f} nm")
            print(f"   採樣點數: {len(positions)}")
            if 'stats' in profile_data:
                stats = profile_data['stats']
                print(f"   平均電流範圍: {stats.get('min_avg_current', 0):.3f} ~ {stats.get('max_avg_current', 0):.3f} pA")
            
            return profile_data
        else:
            print(f"❌ 提取失敗: {result.get('error')}")
            return None
            
    except Exception as e:
        print(f"❌ 錯誤: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

# 測試能帶剖面
if load_success:
    print("📈 測試能帶剖面提取:")
    print("=" * 50)
    
    h, w, _ = cits_data.cits_data.shape
    
    # 水平剖面 (中間)
    print("\n1️⃣ 水平剖面 (中間):")
    profile1 = extract_line_profile((h//2, 0), (h//2, w-1))
    
    # 對角線剖面
    print("\n2️⃣ 對角線剖面:")
    profile2 = extract_line_profile((0, 0), (h-1, w-1))
else:
    print("⚠️ 請先成功載入 CITS 檔案")

## 🎯 步驟 5: 交互式點選和拉線 / Step 5: Interactive Point Selection and Line Drawing

In [ ]:
# 創建交互式選擇工具
profile_plot = widgets.Output()

# 座標輸入
point_x = widgets.IntText(value=50, description='點 X:', min=0)
point_y = widgets.IntText(value=50, description='點 Y:', min=0)

# 線起終點輸入
start_x = widgets.IntText(value=0, description='起點 X:', min=0)
start_y = widgets.IntText(value=50, description='起點 Y:', min=0)
end_x = widgets.IntText(value=100, description='終點 X:', min=0)
end_y = widgets.IntText(value=50, description='終點 Y:', min=0)

# 按鈕
point_btn = widgets.Button(description='提取點能譜', button_style='primary')
line_btn = widgets.Button(description='提取能帶剖面', button_style='success')

def on_point_click(b):
    with profile_plot:
        clear_output(wait=True)
        extract_point_spectrum(point_x.value, point_y.value)

def on_line_click(b):
    with profile_plot:
        clear_output(wait=True)
        extract_line_profile(
            (start_y.value, start_x.value),
            (end_y.value, end_x.value)
        )

point_btn.on_click(on_point_click)
line_btn.on_click(on_line_click)

# 顯示交互式介面
if load_success:
    print("🎯 交互式分析工具:")
    print("=" * 50)
    
    # 顯示當前 CITS 尺寸
    h, w, _ = cits_data.cits_data.shape
    print(f"📏 CITS 影像尺寸: {w} × {h} pixels")
    print(f"📍 座標範圍: X (0-{w-1}), Y (0-{h-1})")
    print()
    
    # 設置最大值
    point_x.max = w - 1
    point_y.max = h - 1
    start_x.max = w - 1
    start_y.max = h - 1
    end_x.max = w - 1
    end_y.max = h - 1
    
    # 點能譜控制
    display(widgets.HTML('<h3>📍 單點能譜分析</h3>'))
    display(widgets.HBox([point_x, point_y, point_btn]))
    
    # 能帶剖面控制
    display(widgets.HTML('<h3>📈 能帶剖面分析</h3>'))
    display(widgets.HBox([start_x, start_y]))
    display(widgets.HBox([end_x, end_y, line_btn]))
    
    # 輸出區域
    display(profile_plot)
else:
    print("⚠️ 請先成功載入 CITS 檔案")

## 📊 步驟 6: 進階分析功能 / Step 6: Advanced Analysis Features

In [ ]:
def show_cits_with_markers(points=None, lines=None):
    """顯示 CITS 影像並標記選擇的點和線"""
    if cits_data is None:
        print("❌ 請先載入 CITS 檔案")
        return
    
    # 使用當前偏壓切片
    slice_data = cits_data.cits_data[:, :, current_bias_index]
    bias_value = cits_data.bias_range[current_bias_index]
    
    fig = go.Figure()
    
    # 添加 CITS 影像
    fig.add_trace(go.Heatmap(
        z=slice_data,
        colorscale='RdBu_r',
        showscale=True,
        colorbar=dict(title='Current (pA)')
    ))
    
    # 添加點標記
    if points:
        for i, (x, y) in enumerate(points):
            fig.add_trace(go.Scatter(
                x=[x], y=[y],
                mode='markers',
                marker=dict(size=10, color='yellow', symbol='x'),
                name=f'點 {i+1}',
                showlegend=True
            ))
    
    # 添加線標記
    if lines:
        for i, ((x1, y1), (x2, y2)) in enumerate(lines):
            fig.add_trace(go.Scatter(
                x=[x1, x2], y=[y1, y2],
                mode='lines+markers',
                line=dict(color='lime', width=2),
                marker=dict(size=8),
                name=f'線 {i+1}',
                showlegend=True
            ))
    
    fig.update_layout(
        title=f'CITS 影像 @ {bias_value:.3f} V (標記點和線)',
        xaxis_title='X (pixels)',
        yaxis_title='Y (pixels)',
        width=700,
        height=700,
        yaxis=dict(scaleanchor="x", scaleratio=1)
    )
    
    fig.show()

# 測試標記功能
if load_success:
    print("🎨 顯示標記的 CITS 影像:")
    
    # 定義一些測試點和線
    test_points = [(50, 50), (100, 100)]
    test_lines = [((0, 50), (150, 50)), ((75, 0), (75, 150))]
    
    show_cits_with_markers(points=test_points, lines=test_lines)

# 計算和顯示 dI/dV map
def calculate_didv_map(bias_index):
    """計算特定偏壓的 dI/dV map"""
    if cits_analyzer is None:
        return None
        
    result = cits_analyzer.calculate_didv_map()
    if result['success']:
        didv_data = result['data']['didv_data']
        return didv_data[:, :, bias_index]
    return None

# 顯示 dI/dV
if load_success:
    print("\n📈 計算 dI/dV map:")
    didv_slice = calculate_didv_map(current_bias_index)
    if didv_slice is not None:
        print(f"✅ dI/dV 計算完成，形狀: {didv_slice.shape}")
    else:
        print("❌ dI/dV 計算失敗")

## 📋 測試總結 / Test Summary

In [ ]:
print("📋 CITS 分析測試總結")
print("=" * 50)

if load_success and cits_data is not None:
    print(f"✅ CITS 檔案載入成功")
    print(f"📊 資料維度: {cits_data.cits_data.shape}")
    print(f"⚡ 偏壓範圍: {cits_data.bias_range[0]:.3f} ~ {cits_data.bias_range[-1]:.3f} V")
    print(f"📏 空間範圍: {cits_data.x_range:.1f} × {cits_data.y_range:.1f} nm")
    print()
    print("🔧 可用功能:")
    print("   ✓ 偏壓切片顯示 (含 slider)")
    print("   ✓ 單點能譜提取")
    print("   ✓ 能帶剖面分析")
    print("   ✓ 交互式點選和拉線")
    print("   ✓ dI/dV 計算")
    print("   ✓ 影像標記功能")
else:
    print("❌ CITS 檔案載入失敗")
    
print("\n💡 使用提示:")
print("1. 使用 bias slider 查看不同偏壓的電流分布")
print("2. 輸入座標提取單點能譜 (I-V 曲線)")
print("3. 指定起終點提取能帶剖面")
print("4. 所有座標使用像素單位 (0 開始)")